# Custom Model

In [11]:
import tensorflow as tf

import keras

from keras import layers
from keras import models
from keras import optimizers

from keras.utils import to_categorical

from sklearn.model_selection import train_test_split




## Dataset Import

In [12]:
cifar10 = tf.keras.datasets.cifar10
(X_train, Y_train), (X_test,Y_test) = cifar10.load_data()

sx_train,sx_val,sy_train,sy_val = train_test_split(X_train, Y_train, test_size = 0.2)

sy_train = to_categorical(sy_train, num_classes = 10)
sy_val   = to_categorical(sy_val, num_classes = 10)
Y_test   = to_categorical(Y_test, num_classes = 10)

sx_train = sx_train * 1.0/255

sx_val = sx_val * 1.0/255

X_test = X_test * 1.0/255

print(sx_train.shape, sx_val.shape, X_test.shape)
print(sy_train.shape, sy_val.shape, Y_test.shape)

(40000, 32, 32, 3) (10000, 32, 32, 3) (10000, 32, 32, 3)
(40000, 10) (10000, 10) (10000, 10)


In [13]:
cmi = keras.Input(shape=(None,None,3))

end_layers = []

#001
# conv_scales = [2, 4, 8, 16]
# inp_shapes =  [18, 9, 6, 3]

#001
# for i in range(len(conv_scales)):
#     conv_scale = conv_scales[i]
#     inp_shape = inp_shapes[i]
#     x = layers.Dropout(0.1)(cmi)
#     x = layers.Conv2D(4 * conv_scale,  (inp_shape,inp_shape), activation='relu', padding='same')(x)
#     x = layers.MaxPooling2D((2,2))(x)
#     x1 = layers.Conv2D(8 * conv_scale,  (3,3), activation='relu')(x)
#     x2 = layers.Conv2D(8 * conv_scale, (3,3), activation='relu', padding='same')(x1)
#     x = layers.add([x1,x2])
#     x = layers.MaxPooling2D((2,2))(x)
#     x1 = layers.Conv2D(64, (3,3), activation='relu')(x)
#     x2 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x1)
#     x = layers.add([x1,x2])
#     x = layers.MaxPooling2D((2,2))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Flatten()(x)
#     end_layers.append(x)


#002
conv_scales = [ 2,  4, 8, 16, 32]
inp_shapes =  [36, 18, 9,  6,  3]
for i in range(len(conv_scales)):
    conv_scale = conv_scales[i]
    inp_shape = inp_shapes[i]
    x = layers.Dropout(0.1)(cmi)
    x = layers.Conv2D(4 * conv_scale,  (inp_shape,inp_shape), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x1 = layers.Conv2D(8 * conv_scale,  (3,3), activation='relu')(x)
    x = layers.MaxPooling2D((2,2))(x1)
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    end_layers.append(x)


cm_concat = layers.concatenate(end_layers)

central_model = models.Model(inputs=cmi, outputs=cm_concat)


wrapper_model = models.Sequential()
wrapper_model.add(layers.InputLayer(input_shape=(32, 32, 3)))
wrapper_model.add(layers.Resizing(224,224, interpolation='bicubic'))
wrapper_model.add(central_model)
wrapper_model.add(layers.Flatten())
wrapper_model.add(layers.Dense(256, activation='relu'))
wrapper_model.add(layers.Dense(10, activation='softmax'))

wrapper_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing_3 (Resizing)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_14 (Functional)      │ (None, 1500400)        │       460,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_20 (Flatten)            │ (None, 1500400)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │   384,102,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 384,565,586 (1.43 GB)

 Trainable params: 384,564,594 (1.43 GB)

 Non-trainable params: 992 (3.88 KB)

In [ ]:
optimizer = optimizers.Adam()

wrapper_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

wrapper_model.fit(sx_train, sy_train, epochs=10, batch_size=int(sx_train.shape[0]/100), validation_data=(sx_val, sy_val))

Epoch 1/10


In [ ]:
wrapper_model.evaluate(X_test, Y_test)

wrapper_model.save('wrapper_model_002.keras')

central_model.save('central_model_002.keras')

313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.2121 - loss: 278.3063
